# The Data Analysis Bureau Exercise

In [29]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random

from statsmodels.tsa.stattools import adfuller, kpss
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from statsmodels.tsa.vector_ar.var_model import VAR
import xgboost as xgb
from sklearn.metrics import classification_report, roc_auc_score
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

## Data import and inspection

In [2]:
# Open JSON file and assign to dict
f = open('/Users/sam/Downloads/DS 2021 Exercise/data_dict.json')
data_dict = json.load(f)
f.close()

In [3]:
# Print line-by-line to fit everything
for key in data_dict.keys():
    print(data_dict[key])

{'name': 'CurrentSpeed', 'units': 'knots', 'range': None, 'description': ' '}
{'name': 'CurrentDir', 'units': 'degrees', 'range': None, 'description': ' '}
{'name': 'TWS', 'units': 'knots', 'range': None, 'description': 'True Wind Speed'}
{'name': 'TWA', 'units': 'degrees', 'range': None, 'description': 'True Wind Angle'}
{'name': 'AWS', 'units': 'knots', 'range': None, 'description': 'Apparent Wind Speed'}
{'name': 'AWA', 'units': 'degrees', 'range': None, 'description': 'Apparent Wind Angle'}
{'name': 'Roll', 'units': 'degrees', 'range': None, 'description': 'Roll, also equals to -Heel'}
{'name': 'Pitch', 'units': 'degrees', 'range': None, 'description': 'Pitch angle'}
{'name': 'HeadingMag', 'units': 'degrees', 'range': None, 'description': 'magnetic heading'}
{'name': 'HoG', 'units': 'degrees', 'range': None, 'description': 'heading over ground'}
{'name': 'HeadingTrue', 'units': 'degrees', 'range': None, 'description': 'true heading. True heading - heading over ground = Yaw'}
{'name

In [3]:
# Import data and inspect top
data = pd.read_csv('/Users/sam/Downloads/test_data.csv')
data.head()

,CurrentSpeed,CurrentDir,TWS,TWA,AWS,AWA,Roll,Pitch,HeadingMag,HoG,...,VMG,RudderAng,Leeway,TWD,WSoG,VoltageDrawn,ModePilote,DateTime,Yaw,Tacking
0,0.0756,123.0,10.8,48.0,10.4,48.0,-3.54,9.080000,24.0,308.0,...,0.0594,4.666667,0.0,356.0,10.5,11.8,5.0,2019-04-14 00:00:00.000,-299.0,1.0
1,0.0756,123.0,10.8,48.0,10.4,48.0,-3.54,9.080000,24.0,308.0,...,0.0594,4.666667,0.0,356.0,10.5,11.8,5.0,2019-04-14 00:00:01.000,-299.0,1.0
2,0.0756,123.0,10.8,48.0,10.4,48.0,-3.52,9.099999,24.0,308.0,...,0.0594,4.666667,0.0,356.0,9.9,11.8,5.0,2019-04-14 00:00:02.000,-299.0,1.0
3,0.0756,123.0,10.8,48.0,10.4,48.0,-3.52,9.099999,24.0,308.0,...,0.0594,4.666667,0.0,356.0,9.9,11.8,5.0,2019-04-14 00:00:03.000,-299.0,1.0
4,0.0756,123.0,10.8,48.0,10.4,48.0,-3.50,9.099999,24.0,308.0,...,0.0594,4.666667,0.0,356.0,10.3,11.8,5.0,2019-04-14 00:00:04.000,-299.0,1.0


In [4]:
# Convert date to DT
data['DateTime'] = pd.to_datetime(data['DateTime'])

In [4]:
# Check column types
data.info()

TypeError: Cannot interpret '<attribute 'dtype' of 'numpy.generic' objects>' as a data type

In [20]:
# Describe numerical data
data.iloc[:,12:].describe()

,Longitude,Latitude,SoG,SoS,AvgSoS,VMG,RudderAng,Leeway,TWD,WSoG,VoltageDrawn,ModePilote,Yaw,Tacking
count,219836.000000,219840.000000,219842.000000,219840.000000,219838.000000,219837.000000,219838.000000,219839.000000,219838.000000,219836.000000,219839.000000,219839.000000,219834.000000,219995.000000
mean,-60.675999,16.805625,7.658572,7.607856,6.382535,4.109238,2.025093,-1.226548,83.323893,14.746686,12.417475,2.422614,5.595763,0.209273
std,0.982475,3.929849,3.075285,3.084592,2.903147,2.066419,4.963518,0.793367,53.810225,4.291340,0.570748,1.043669,140.521400,0.406791
min,-61.816873,11.971172,0.005400,0.000000,0.070200,0.000000,-37.333336,-10.000000,0.000000,0.000000,11.100000,2.000000,-359.000000,0.000000
25%,-61.639917,12.913855,7.684200,7.597800,5.637600,3.148200,-0.666667,-2.000000,61.000000,11.800000,12.100000,2.000000,-13.000000,0.000000
50%,-61.199546,15.232683,8.726400,8.699400,7.581600,4.579200,2.333333,-1.000000,70.000000,14.800000,12.300000,2.000000,-6.000000,0.000000
75%,-59.735390,20.902214,9.369000,9.298800,8.461801,5.583600,4.666667,-1.000000,82.000000,17.600000,12.500000,2.000000,-1.000000,0.000000
max,-59.279375,22.209945,12.598201,12.700800,8.532001,9.860400,47.000000,9.000000,359.000000,35.700001,14.200000,5.000000,359.000000,1.000000


In [22]:
for col in data.columns:
    print(f'{col}: {data[col].isna().sum()}')

CurrentSpeed: 0
CurrentDir: 0
TWS: 0
TWA: 0
AWS: 0
AWA: 0
Roll: 0
Pitch: 0
HeadingMag: 0
HoG: 0
HeadingTrue: 0
AirTemp: 0
Longitude: 0
Latitude: 0
SoG: 0
SoS: 0
AvgSoS: 0
VMG: 0
RudderAng: 0
Leeway: 0
TWD: 0
WSoG: 0
VoltageDrawn: 0
ModePilote: 0
DateTime: 0
Yaw: 0
Tacking: 0


## Cleaning data

Interpolate NaN in DT as we know it samples once per second. So if DT is NaN at index 1, we can insert the value as DT at index 0 plus one second.

In [5]:
# Get index of missing in DT
inds = pd.isnull(data['DateTime']).to_numpy().nonzero()[0]
inds

array([ 39959,  81738,  82751,  91468, 121548])

In [6]:
# For each missing value, we add the previous timestamp plus 1
# Note that we run chronologically down DF
# Thus, if there are multiple NaN in a row (not the case), this would be fine
for item in inds:
    data.loc[item, 'DateTime'] = data.loc[item-1, 'DateTime'] + pd.to_timedelta(1, unit='s')

Other missing values could be interpolated, as we might assume that things such as direction and speed might not change drastically from one time point to another. This could be done by taking an average of the n previous and following values and setting it in place of a missing value. This would exclude the Tacking and ModePilote columns as they seem categorical.

In [7]:
# Interpolating for all other numerical variables
# Input mean of previous 5 and following 5 values
for col in [item for item in data.columns if item not in ['ModePilote', 'Tacking']]:
    inds = pd.isnull(data[col]).to_numpy().nonzero()[0]
    for i in inds:
           data.loc[i, col] = data.loc[(i-5):(i+5), col].mean()

# For categorical variables, we take only the previous value
# Trade-off between labels being correct and complete dataset
for col in ['ModePilote', 'Tacking']:
    inds = pd.isnull(data[col]).to_numpy().nonzero()[0]
    for i in inds:
           data.loc[i, col] = data.loc[i-1, col].mean()

In [8]:
# Scale numeric features
numeric_cols = [item for item in data.columns if item not in ['ModePilote', 'Tacking', 'DateTime']]
X_numeric = data[numeric_cols]

In [10]:
# Check if data is stationary

# Augmented Dickey-Fuller Test (ADF Test)/unit root test
def adf_test(ts, signif=0.05):
    dftest = adfuller(ts, autolag='AIC')
    adf = pd.Series(dftest[0:4], index=['Test Statistic','p-value','# Lags','# Observations'])
    for key,value in dftest[4].items():
       adf['Critical Value (%s)'%key] = value
    #print(adf)
    
    p = adf['p-value']
    if p > signif:
        print(f'Series is Non-Stationary')

# Look at all numerical features
for col in X_numeric.columns:
    print(col)
    adf_test(X_numeric[col])

CurrentSpeed
CurrentDir
TWS
TWA
AWS
AWA
Roll
Pitch
HeadingMag
HoG
HeadingTrue
AirTemp
Longitude
 Series is Non-Stationary
Latitude
 Series is Non-Stationary
SoG
SoS
AvgSoS
VMG
RudderAng
Leeway
TWD
WSoG
VoltageDrawn
Yaw


In [17]:
# KPSS
def kpss_test(ts):
    kpsstest = kpss(ts, regression='c', lags='auto')
    kpss_output = pd.Series(kpsstest[0:3], index=['Test Statistic','p-value','Lags Used'])
    if kpss_output['p-value'] > .05:
        print('Stationary')

# Look at all numerical features
for col in X_numeric.columns:
    print(col)
    kpss_test(X_numeric[col])

CurrentSpeed
CurrentDir


/Users/sam/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:1708: InterpolationWarning: p-value is smaller than the indicated p-value
  warn("p-value is smaller than the indicated p-value", InterpolationWarning)
/Users/sam/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:1708: InterpolationWarning: p-value is smaller than the indicated p-value
  warn("p-value is smaller than the indicated p-value", InterpolationWarning)
/Users/sam/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:1708: InterpolationWarning: p-value is smaller than the indicated p-value
  warn("p-value is smaller than the indicated p-value", InterpolationWarning)


TWS
TWA


/Users/sam/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:1708: InterpolationWarning: p-value is smaller than the indicated p-value
  warn("p-value is smaller than the indicated p-value", InterpolationWarning)
/Users/sam/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:1708: InterpolationWarning: p-value is smaller than the indicated p-value
  warn("p-value is smaller than the indicated p-value", InterpolationWarning)


AWS
AWA


...

/Users/sam/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:1708: InterpolationWarning: p-value is smaller than the indicated p-value
  warn("p-value is smaller than the indicated p-value", InterpolationWarning)
/Users/sam/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:1708: InterpolationWarning: p-value is smaller than the indicated p-value
  warn("p-value is smaller than the indicated p-value", InterpolationWarning)
/Users/sam/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:1708: InterpolationWarning: p-value is smaller than the indicated p-value
  warn("p-value is smaller than the indicated p-value", InterpolationWarning)
/Users/sam/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:1708: InterpolationWarning: p-value is smaller than the indicated p-value
  warn("p-value is smaller than the indicated p-value", InterpolationWarning)


HeadingTrue
AirTemp
Longitude
Latitude


/Users/sam/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:1708: InterpolationWarning: p-value is smaller than the indicated p-value
  warn("p-value is smaller than the indicated p-value", InterpolationWarning)
/Users/sam/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:1708: InterpolationWarning: p-value is smaller than the indicated p-value
  warn("p-value is smaller than the indicated p-value", InterpolationWarning)
/Users/sam/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:1708: InterpolationWarning: p-value is smaller than the indicated p-value
  warn("p-value is smaller than the indicated p-value", InterpolationWarning)
/Users/sam/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:1708: InterpolationWarning: p-value is smaller than the indicated p-value
  warn("p-value is smaller than the indicated p-value", InterpolationWarning)
/Users/sam/opt/anaconda3/lib/python3.7/site-packages/statsmodels

SoG
SoS
AvgSoS
VMG
RudderAng
Leeway
TWD
WSoG
VoltageDrawn
Yaw


/Users/sam/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:1708: InterpolationWarning: p-value is smaller than the indicated p-value
  warn("p-value is smaller than the indicated p-value", InterpolationWarning)
/Users/sam/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:1708: InterpolationWarning: p-value is smaller than the indicated p-value
  warn("p-value is smaller than the indicated p-value", InterpolationWarning)
/Users/sam/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:1708: InterpolationWarning: p-value is smaller than the indicated p-value
  warn("p-value is smaller than the indicated p-value", InterpolationWarning)
/Users/sam/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:1708: InterpolationWarning: p-value is smaller than the indicated p-value
  warn("p-value is smaller than the indicated p-value", InterpolationWarning)


#### RESULTS stationarity
- ADF non-stationary: longitude, latitude
- KPSS non-stationary: All variables
- Longitude & latitude non-stationary
- All others difference stationary

In [9]:
# Difference all numeric variables
for col in X_numeric.columns:
    X_numeric[col] = X_numeric[col] - X_numeric[col].shift(1)

X_numeric = X_numeric.dropna()

/Users/sam/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
# Scale numeric features
scaled_features = StandardScaler().fit_transform(X_numeric.values)
scaled_df = pd.DataFrame(scaled_features, columns=X_numeric.columns)

In [11]:
# Dimensionality reduction
array_x_pca = np.array(scaled_df)

# Create the PCA instance
pca = PCA(n_components = 0.95)

# Fit on data
pca.fit(array_x_pca)

# Access values and vectors
print(pca.explained_variance_)

# Transform data
smol_scaled = pca.transform(array_x_pca)

[3.70173918 2.92720279 2.71206686 1.89709625 1.64628745 1.28756367
 1.0596012  1.00454775 0.99968694 0.98959534 0.98099261 0.96942611
 0.92702086 0.77756077 0.61481912 0.49956376]


In [12]:
smol_scaled2 = pd.DataFrame(smol_scaled)

In [13]:
# Change ModePilote to 0 and 1
data.loc[data['ModePilote'] == 5, 'ModePilote'] = 1
data.loc[data['ModePilote'] == 2, 'ModePilote'] = 0

In [14]:
# Combined numeric and ModePilote
cat_df = data.loc[1:, ['ModePilote', 'Tacking']]
cat_df.index = range(219999)
full_df = pd.concat([smol_scaled2, cat_df], axis=1, ignore_index=True)

In [15]:
# Function from https://machinelearningmastery.com/convert-time-series-supervised-learning-problem-python/

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	"""
	Frame a time series as a supervised learning dataset.
	Arguments:
		data: Sequence of observations as a list or NumPy array.
		n_in: Number of lag observations as input (X).
		n_out: Number of observations as output (y).
		dropnan: Boolean whether or not to drop rows with NaN values.
	Returns:
		Pandas DataFrame of series framed for supervised learning.
	"""
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg
 

In [43]:
# Make supervised df
supervised_df = series_to_supervised(full_df.values, 2)

In [44]:
# Split into train and test sets
values = supervised_df.values
n_train_hours = int(round(values.shape[0]*0.7, 0))
train = values[:n_train_hours, :]
test = values[n_train_hours:, :]

# Split into input and outputs
X_train, y_train = train[:, :-1], train[:, -1]
X_test, y_test = test[:, :-1], test[:, -1]

## XGBoost

In [45]:
# Make model
xgb_mod = xgb.XGBClassifier(random_state=47, eta=0.2, min_child_weight=5, gamma=0.1,
subsample=0.8, colsample_bytree=0.8, max_depth=8, reg_alpha=10, scale_pos_weight=0.3)

xgb_mod.fit(X_train, y_train)

XGBClassifier(colsample_bytree=0.8, eta=0.2, gamma=0.1, max_depth=8,
              min_child_weight=5, random_state=47, reg_alpha=10,
              scale_pos_weight=0.3, subsample=0.8)

In [46]:
y_pred = xgb_mod.predict(X_test)
print(classification_report(y_test, y_pred))
print(roc_auc_score(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     60479
         1.0       1.00      1.00      1.00      5520

    accuracy                           1.00     65999
   macro avg       1.00      1.00      1.00     65999
weighted avg       1.00      1.00      1.00     65999

0.9999011529573926


## LSTM

In [24]:
# Reshape input to be 3D [samples, timesteps, features]
X_train_nn = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_nn = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
print(X_train_nn.shape, y_train.shape, X_test_nn.shape, y_test.shape)


(153998, 1, 53) (153998,) (65999, 1, 53) (65999,)


In [25]:
# Build network
model = Sequential()
model.add(LSTM(50, input_shape=(X_train_nn.shape[1], X_train_nn.shape[2])))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam')

# Fit network
history = model.fit(X_train_nn, y_train, epochs=5, batch_size=72, validation_data=(X_test, y_test), verbose=2, shuffle=False)

Train on 153998 samples, validate on 65999 samples
Epoch 1/5
 - 144s - loss: 0.0898 - val_loss: 0.0137
Epoch 2/5
 - 90s - loss: 0.0085 - val_loss: 0.0026
Epoch 3/5
 - 100s - loss: 0.0020 - val_loss: 0.0012
Epoch 4/5
 - 115s - loss: 0.0013 - val_loss: 7.1605e-04
Epoch 5/5
 - 55s - loss: 0.0010 - val_loss: 5.4979e-04


In [26]:
# make a prediction
y_pred = model.predict(X_test)

array([[1.4722347e-04],
       [6.0707331e-05],
       [1.1425614e-03],
       [6.5714121e-05],
       [1.5813112e-04],
       [3.2490492e-04],
       [6.1988831e-05],
       [8.4131956e-05],
       [4.5657158e-05],
       [1.0409951e-04]], dtype=float32)

In [33]:
y_pred_bin = [1 if item > 0.5 else 0 for item in y_pred]
print(classification_report(y_test, y_pred_bin))
print(roc_auc_score(y_test, y_pred_bin))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     60479
         1.0       1.00      1.00      1.00      5520

    accuracy                           1.00     65999
   macro avg       1.00      1.00      1.00     65999
weighted avg       1.00      1.00      1.00     65999

0.9997609692524726


In [ ]:
# Choose forecasting horizon (e.g. 15-30s)
forecasting_horizon = 15

# This is only if we use all data to predict final bit
# Look into rolling prediction and evaluation
train = data.head(-forecasting_horizon).copy()
y_valid = data.tail(forecasting_horizon)['y']

# Include lags in dataset (could be more/less/different)
train['lag-15'] = train['y'].shift(15)
train['lag-30'] = train['y'].shift(30)

# drop NAs
train_without_nulls = train.dropna()
X_train = train_without_nulls.drop(columns='y')
y_train = train_without_nulls['y']

In [ ]:
# VAR model
model = VAR(endog=train)

In [ ]:
# XGBoost

# Walk-forward evaluation